# PREPARATIONS NEEDED TO RUN JOAKIMS CODE

In [1]:
from datetime import timedelta
import pandas as pd
import numpy as np

In [2]:
dtype_dict = {
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'AnimalNumber': 'Int64',
    'Del_Cow_Id': 'Int64',
    'StartDate': 'str',
    'StartTime': 'str',
    'CalvingDate': 'str',
    'LactationNumber': 'Int64',
    'DaysInMilk': 'Int64',
    'YearSeason': 'str',
    'TotalYield': 'float',
    'Breed': 'str',
    'Temperature': 'float',
    'RelativeHumidity': 'float',
    'THI_adj': 'float',
    'HW': 'Int64',
    'cum_HW': 'Int64',
    'Temp15Threshold': 'Int64'
}

#
#
# Make YS
df = pd.read_csv('../Data/MY_weather.csv', low_memory=False, dtype=dtype_dict)
print(df.shape)
df['StartDate'] = pd.to_datetime(df['StartDate'])


# Function to determine the season based on month
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    elif month in [9, 10, 11]:
        return 4


# Apply the function to create a 'YearSeason' variable
df['YearSeason'] = df['StartDate'].apply(lambda x: f"{x.year}0{get_season(x)}")

# Save the updated dataframe to CSV
col_keep = ["FarmName_Pseudo", 'SE_Number', 'AnimalNumber', 'Del_Cow_Id', 'Breed', 'CalvingDate', 'LactationNumber',
            'DaysInMilk', 'StartDate', 'StartTime', 'TotalYield', 'YearSeason',
            'Temperature', 'RelativeHumidity', 'THI_adj', 'HW', 'cum_HW', 'Temp15Threshold']
df = df[col_keep]


(1868019, 51)


In [3]:
# ADD PARENTAGE
df2 = pd.read_csv("../Data/updateDF.csv", low_memory=False)
col_keep = ["SE_Number", "BirthDate", "Father_SE_Number", "Mother_SE_Number", "CullingDate"]
df2 = df2[col_keep]
df2 = df2.drop_duplicates(subset=["SE_Number"])

df3 = pd.merge(df, df2, on="SE_Number", how="left")

In [4]:
# MAKE AGE
df3['BirthDate'] = pd.to_datetime(df3['BirthDate'])
df3['StartDate'] = pd.to_datetime(df3['StartDate'])
df3['Age'] = (df3['StartDate'] - df3['BirthDate']).dt.days

df3.rename(columns={"CullingDate": "CullDecisionDate", "Father_SE_Number": "Father", "Mother_SE_Number": "Mother", "Breed": "BreedName"}, 
           inplace=True)

In [6]:
# Keep only MY data from 2022-01-01 until 2024-08-18
cutoff_date1 = '2022-01-01'
cutoff_date2 = '2024-08-18'
df3 = df3[df3['StartDate'] >= cutoff_date1]
df3 = df3[df3['StartDate'] <= cutoff_date2]

missing_count = df3['THI_adj'].isna().sum()
print(f"Number of missing observations in 'THI_adj': {missing_count}")  # 607,201
print(df3.shape)  # 1,379,231

df3.to_csv("../Data/UpdateMY_weather.csv", index=False)

Number of missing observations in 'THI_adj': 607201
(1379231, 23)
